## 1
- ✅ Lệnh này giúp cài đặt hoặc cập nhật thư viện openai và datasets trong môi trường làm việc.
- ✅ -qU giúp quá trình cài đặt diễn ra nhanh gọn mà không hiển thị quá nhiều thông tin.
- ✅ Dùng được trên Google Colab, Jupyter Notebook hoặc môi trường terminal (bỏ !).


In [41]:
# Cài đặt thư viện cần thiết
! pip install -qU openai datasets  


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## 2
- ✅ getpass: Dùng để nhập thông tin nhạy cảm như API key mà không hiển thị trên màn hình.
- ✅ openai: Thư viện để gọi API GPT-3.5, GPT-4 từ OpenAI.

In [4]:
import getpass  # Thư viện giúp nhập mật khẩu hoặc API key một cách bảo mật
from openai import OpenAI  # Thư viện OpenAI để gọi API GPT

In [42]:
import os  # Thư viện cung cấp các chức năng tương tác với hệ điều hành

# Nhắc người dùng nhập API key một cách bảo mật (không hiển thị khi nhập)
os.environ['OPENAI_API_KEY'] = getpass.getpass('Lấy API key nhập vào đây: ')  

## 3
- ✅ load_dataset("gaodrew/midjourney-prompts-highquality", split="train")
	•	Tải dataset từ Hugging Face (gaodrew/midjourney-prompts-highquality).
	•	Chỉ lấy phần dữ liệu train.

- ✅ dataset.select(range(1000))
	•	Chọn 1000 mẫu đầu tiên từ tập dữ liệu.
	•	Giúp giảm dung lượng khi xử lý dữ liệu lớn.

- Lợi ích:

✔️ Nhanh hơn: Chỉ lấy 1000 mẫu thay vì toàn bộ dataset.
✔️ Dễ kiểm soát: Giúp kiểm tra dữ liệu trước khi xử lý toàn bộ.

In [25]:
from datasets import load_dataset  # Import hàm load_dataset từ thư viện datasets (Hugging Face)

# Tải tập dữ liệu "midjourney-prompts-highquality" từ Hugging Face
# Tham số "split='train'" chỉ tải phần dữ liệu huấn luyện
dataset = load_dataset("gaodrew/midjourney-prompts-highquality", split="train")

# Chọn 1000 mẫu đầu tiên từ tập dữ liệu để sử dụng
selected_dataset = dataset.select(range(1000))  

## 4
- 🔹 Chức năng của code:
-	•	Trích xuất nội dung “Prompt” từ tập dữ liệu selected_dataset.
-	•	Lưu danh sách các prompt vào prompt_pairs dưới dạng danh sách từ điển.
-	•	Dùng biểu thức chính quy (re) để tìm các đoạn văn bản bắt đầu bằng "Prompt: " và kết thúc bằng dấu xuống dòng (\n).

In [64]:
import re  # Thư viện hỗ trợ xử lý chuỗi bằng biểu thức chính quy (regex)

# Lấy danh sách nội dung từ cột 'content' trong dataset đã chọn
enhanced_prompts_raw = selected_dataset['content']

# Danh sách chứa các prompt sau khi trích xuất
prompt_pairs = []

def extract_prompts(content):
    """
    Hàm trích xuất nội dung sau từ "Prompt: " trong chuỗi văn bản.
    
    Args:
        content (str): Chuỗi văn bản đầu vào chứa prompt.
    
    Returns:
        str: Nội dung prompt được trích xuất hoặc None nếu không tìm thấy.
    """
    match = re.search(r"\*\*(.*?)\*\*", text)  # Tìm đoạn văn bản sau "Prompt: " đến ký tự xuống dòng
    
    if match:
        extracted_prompt = match.group(1)
        return extracted_prompt  # Lấy nội dung tìm thấy từ regex
    return None  # Trả về None nếu không có prompt nào được tìm thấy

# Lặp qua từng đoạn văn bản trong tập dữ liệu và trích xuất prompt
for text in enhanced_prompts_raw:
    prompt_pairs.append({
        "enhanced_prompts": extract_prompts(text)  # Lưu nội dung prompt vào danh sách
    })

 ## 5
- 🔹 Chức năng của chương trình:
-	•	Gọi OpenAI API để tạo phiên bản đơn giản của các prompt đã trích xuất.
-	•	Tự động retry nếu gặp lỗi API (dùng tenacity).
-	•	Hiển thị tiến trình khi xử lý nhiều prompt (dùng tqdm).
-	•	Lưu kết quả vào file JSON để sử dụng sau này.
-	•	(Tùy chọn) Tải file JSON về máy nếu chạy trên Google Colab.

In [75]:
# Import các thư viện cần thiết
import json  # Xử lý dữ liệu JSON
import time  # Hỗ trợ delay giữa các request
from openai import OpenAI  # Gọi API OpenAI
from tqdm.auto import tqdm  # Hiển thị tiến trình chạy
from tenacity import retry, stop_after_attempt, wait_exponential  # Hỗ trợ retry khi gặp lỗi

# Khởi tạo OpenAI client
client = OpenAI()

# --------------------------------------
# 🔹 HÀM GỌI API OPENAI VỚI CƠ CHẾ RETRY
# --------------------------------------
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=2, min=5, max=60))
def get_simple_prompt(enhanced_prompt):
    """
    Gọi OpenAI API để đơn giản hóa một prompt đã trích xuất.

    Args:
        enhanced_prompt (str): Prompt cần đơn giản hóa.

    Returns:
        str: Phiên bản đơn giản hóa của prompt.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Sử dụng model gpt-3.5-turbo
        messages=[
            {"role": "system", "content": "You are a customer service representative helping a customer with a problem."},
            {"role": "user", "content": f"ENHANCE PROMPT: {enhanced_prompt}; SIMPLE BASIC PROMPT: "}
        ]
    )
    
    return response.choices[0].message.content

# --------------------------------------
# 🔹 DANH SÁCH PROMPT ĐẦU VÀO
# --------------------------------------
prompt_pairs = [
    {"extracted_prompts": "How can I reset my password?"},
    {"extracted_prompts": "What are your refund policies?"},
    {"extracted_prompts": "How do I update my account details?"},
    {"extracted_prompts": "Where can I find my order history?"}
]

# --------------------------------------
# 🔹 XỬ LÝ VÀ GỌI API
# --------------------------------------
for prompt in tqdm(prompt_pairs):
    try:
        # Gọi API để lấy phiên bản đơn giản của prompt
        prompt['simple_prompt'] = get_simple_prompt(prompt['extracted_prompts'])
        time.sleep(2)  # Chờ 2 giây để tránh quá tải API
    except Exception as e:
        prompt['simple_prompt'] = f"Error: {str(e)}"  # Lưu lỗi nếu gặp vấn đề

# --------------------------------------
# 🔹 LƯU DỮ LIỆU VÀO FILE JSON
# --------------------------------------
output_file = "simple_prompts.json"

with open(output_file, 'w') as f:
    json.dump(prompt_pairs, f, indent=4)  # Lưu file với format đẹp

print(f"✅ Dữ liệu đã được lưu vào {output_file}")

# --------------------------------------
# 🔹 (TÙY CHỌN) TẢI FILE JSON VỀ MÁY TRÊN GOOGLE COLAB
# --------------------------------------
# Nếu chạy trên Google Colab, có thể tải file xuống bằng đoạn code sau:
# from google.colab import files
# files.download(output_file)

  0%|          | 0/4 [00:00<?, ?it/s]

✅ Dữ liệu đã được lưu vào simple_prompts.json


## 6 📌 Mô tả chức năng của chương trình

- Chương trình này giúp quản lý dữ liệu huấn luyện bằng cách:
-	1.	Lưu dữ liệu vào file JSON Lines (.jsonl) → Mỗi dòng là một JSON object.
-	2.	Tải dữ liệu từ file .jsonl nếu đã có dữ liệu trước đó.
-	3.	Thêm dữ liệu mới vào file mà không ghi đè toàn bộ dữ liệu cũ.

In [76]:
# Import các thư viện cần thiết
import json  # Xử lý dữ liệu JSON
from typing import List, Dict  # Xác định kiểu dữ liệu đầu vào và đầu ra cho các hàm

# Định nghĩa đường dẫn file lưu trữ dữ liệu
FILE_PATH = "training_data.jsonl"

# ------------------------------
# 🔹 HÀM LƯU DỮ LIỆU VÀO FILE JSONL
# ------------------------------

def save_training_data(data: List[Dict], output_file: str = FILE_PATH):
    """
    Ghi dữ liệu vào file JSON Lines (.jsonl)
    
    Args:
        data (List[Dict]): Danh sách dữ liệu dạng dictionary cần lưu
        output_file (str): Đường dẫn file lưu trữ
    """
    with open(output_file, 'a') as f:  # Mở file ở chế độ 'append' (thêm vào cuối file)
        for item in data:
            f.write(json.dumps(item) + "\n")  # Ghi từng object JSON trên 1 dòng

# ------------------------------
# 🔹 HÀM TẢI DỮ LIỆU TỪ FILE JSONL
# ------------------------------

def load_training_data(file_path: str = FILE_PATH) -> List[Dict]:
    """
    Đọc dữ liệu từ file JSONL nếu file tồn tại

    Args:
        file_path (str): Đường dẫn file JSONL

    Returns:
        List[Dict]: Danh sách dữ liệu đã đọc từ file
    """
    try:
        with open(file_path, 'r') as f:  # Mở file ở chế độ đọc ('r')
            return [json.loads(line) for line in f]  # Đọc từng dòng, chuyển thành JSON object
    except (FileNotFoundError, json.JSONDecodeError):  
        return []  # Nếu file không tồn tại hoặc có lỗi định dạng, trả về danh sách rỗng

# ------------------------------
# 🔹 HÀM THÊM DỮ LIỆU MỚI VÀO FILE
# ------------------------------

def add_training_data(new_data: List[Dict]):
    """
    Thêm dữ liệu mới vào file JSONL mà không ghi đè dữ liệu cũ

    Args:
        new_data (List[Dict]): Danh sách dữ liệu mới cần thêm
    """
    existing_data = load_training_data()  # Đọc dữ liệu cũ
    existing_data.extend(new_data)  # Thêm dữ liệu mới vào danh sách
    save_training_data(new_data)  # Chỉ ghi dữ liệu mới, không ghi lại toàn bộ file

# ------------------------------
# 🔹 VÍ DỤ SỬ DỤNG
# ------------------------------

prompt_pairs = [
    {"question": "How to reset my password?", "answer": "Go to settings and click reset password."},
    {"question": "What is the refund policy?", "answer": "You can request a refund within 30 days."}
]

# Gọi hàm để thêm dữ liệu mới vào file
add_training_data(prompt_pairs)

In [74]:
example_data = prompt_pairs[0]  # Lấy phần tử đầu tiên từ danh sách prompt_pairs

# Kiểm tra xem các key có tồn tại không trước khi truy xuất
simple_prompt = example_data.get("simple_prompt", "Default simple prompt")  # Dùng `.get()` để tránh lỗi KeyError
enhanced_prompt = example_data.get("enhanced_prompt", "Default enhanced prompt")

# Tạo danh sách tin nhắn
message = [{
            "role": "system",
            "content": "You are a customer service representative helping a customer with a problem."
          },{
            "role": "user", 
            "content": simple_prompt  # Sửa đúng key
            },{
            "role": "assistant",
            "content": enhanced_prompt  # Sửa đúng key
            }]

message  # Hiển thị kết quả

[{'role': 'system',
  'content': 'You are a customer service representative helping a customer with a problem.'},
 {'role': 'user', 'content': 'Default simple prompt'},
 {'role': 'assistant', 'content': 'Default enhanced prompt'}]